In [ ]:
# Table 1, 2 and figure 6 are based on the below result 




In [1]:
# some useful functions for metrics calculation

def softmax(x):
    e_x = np.exp(x - np.max(x,1)[:,np.newaxis])
    return e_x/e_x.sum(1)[:,np.newaxis]

def recall(label_true,label_pred,k):  # Recall@k
    '''the previous two inputs are 2 dimensional array
    '''
    idx = np.argpartition(label_pred, -k, 1)[:,-k:]
    r = label_true[np.repeat(np.arange(label_true.shape[0]),k),idx.flatten()].reshape(-1,k)
    recall = (np.sum(r,1)/np.sum(label_true,1)).mean()
    return recall

def dcg_score(y_true, y_score, k, gains="exponential"):

    order = np.argsort(y_score,axis = 1)[:,::-1][:,:k]
    
    y_t = y_true[np.repeat(np.arange(y_true.shape[0]),k),order.flatten()].reshape(-1,k)

    if gains == "exponential":
        gains = 2 ** y_t - 1
    elif gains == "linear":
        gains = y_t
    else:
        raise ValueError("Invalid gains option.")
    discounts = np.log2(np.arange(len(y_t[0,:])) + 2)[np.newaxis,:]
    dcg = np.sum(gains / discounts,1)
    return dcg

def ndcg_score(y_true, y_score, k, gains="exponential"): # Normalized Discounted Cummulative Gain
   
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_score, k, gains)
    return (actual / best).mean()


In [1]:
#  FCN-4 + output  (MTAT)
import numpy as np 
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 30
input_shape = (96,1360,1)
model = Sequential()

model.add(Conv2D(128,(3,3),padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,4),padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(384,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,5),strides=(2,4),padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(768,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,8),strides=(2,4),padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(2048,(3,3),strides=(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,8),strides=(4,8),padding='same'))
model.add(Dropout(0.5))

model.add(Flatten())
# model.add(Dense(50, activation='sigmoid'))
model.add(Dense(50))
model.add(Activation('sigmoid'))

model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

filepath='C:/DT/model_save/FCN/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_val = np.load(path+'\\x_validation_MelSpec.npy')
y_val = np.load(path+'\\y_validation.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.aucs_all = []
        self.recalls_1 = []
        self.recalls_3 = []
        self.recalls_5 = []
        self.recalls_10 = []
        self.ndcgs_50 = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        
        recall_1 = recall(y_test,y_pred,1)
        recall_3 = recall(y_test,y_pred,3)
        recall_5 = recall(y_test,y_pred,5)
        recall_10 = recall(y_test,y_pred,10)
        self.recalls_1.append(recall_1)
        self.recalls_3.append(recall_3)
        self.recalls_5.append(recall_5)
        self.recalls_10.append(recall_10)
        
        ndcg_50 = ndcg_score(y_test,y_pred,k=50,gains="exponential")
        self.ndcgs_50.append(ndcg_50)
        
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        auc_all = roc_auc_score(y_test, y_pred,average = None )
        self.aucs_all.append(auc_all)
        
        print("Average AUC:",auc)
        print("Recalls:",recall_1,recall_3,recall_5,recall_10)
        print("NDCG:",ndcg_50)
        print("AUCs:",auc_all)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, batch_size = 16, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'fcn_aucs.pkl'),'wb'))
pickle.dump(histories.aucs_all,open(os.path.join(path,'fcn_aucs_all.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'fcn_losses.pkl'),'wb'))
pickle.dump(histories.recalls_1,open(os.path.join(path,'fcn_recalls_1.pkl'),'wb'))
pickle.dump(histories.recalls_3,open(os.path.join(path,'fcn_recalls_3.pkl'),'wb'))
pickle.dump(histories.recalls_5,open(os.path.join(path,'fcn_recalls_5.pkl'),'wb'))
pickle.dump(histories.recalls_10,open(os.path.join(path,'fcn_recalls_10.pkl'),'wb'))
pickle.dump(histories.ndcgs_50,open(os.path.join(path,'fcn_ndcgs_10.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 1360, 128)     1280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 128)     512       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 128)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 340, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 384)      442752    
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 384)      1536      
__________

In [2]:
#  Timbre CNN  (MTAT)
import numpy as np 
from keras.layers import Input,Conv2D,BatchNormalization,Activation,MaxPooling2D,Concatenate,Flatten,Dropout,Dense
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

epochs = 30
pool_size = (4,4)
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1_10_96_1 = Conv2D(10,(96,1),padding='same')(input_melS)
bn1_10_96_1 = BatchNormalization()(conv1_10_96_1)
act1_10_96_1 = Activation('relu')(bn1_10_96_1)
max1_10_96_1 = MaxPooling2D(pool_size,padding='same')(act1_10_96_1)
conv1_6_96_3 = Conv2D(6,(96,3),padding='same')(input_melS)
bn1_6_96_3 = BatchNormalization()(conv1_6_96_3)
act1_6_96_3 = Activation('relu')(bn1_6_96_3)
max1_6_96_3 = MaxPooling2D(pool_size,padding='same')(act1_6_96_3)
conv1_3_96_5 = Conv2D(3,(96,5),padding='same')(input_melS)
bn1_3_96_5 = BatchNormalization()(conv1_3_96_5)
act1_3_96_5 = Activation('relu')(bn1_3_96_5)
max1_3_96_5 = MaxPooling2D(pool_size,padding='same')(act1_3_96_5)
conv1_3_96_7 = Conv2D(3,(96,7),padding='same')(input_melS)
bn1_3_96_7 = BatchNormalization()(conv1_3_96_7)
act1_3_96_7 = Activation('relu')(bn1_3_96_7)
max1_3_96_7 = MaxPooling2D(pool_size,padding='same')(act1_3_96_7)
conv1_15_75_1 = Conv2D(15,(75,1),padding='same')(input_melS)
bn1_15_75_1 = BatchNormalization()(conv1_15_75_1)
act1_15_75_1 = Activation('relu')(bn1_15_75_1)
max1_15_75_1 = MaxPooling2D(pool_size,padding='same')(act1_15_75_1)
conv1_10_75_3 = Conv2D(6,(75,3),padding='same')(input_melS)
bn1_10_75_3 = BatchNormalization()(conv1_10_75_3)
act1_10_75_3 = Activation('relu')(bn1_10_75_3)
max1_10_75_3 = MaxPooling2D(pool_size,padding='same')(act1_10_75_3)
conv1_5_75_5 = Conv2D(3,(75,5),padding='same')(input_melS)
bn1_5_75_5 = BatchNormalization()(conv1_5_75_5)
act1_5_75_5 = Activation('relu')(bn1_5_75_5)
max1_5_75_5 = MaxPooling2D(pool_size,padding='same')(act1_5_75_5)
conv1_5_75_7 = Conv2D(3,(75,7),padding='same')(input_melS)
bn1_5_75_7 = BatchNormalization()(conv1_5_75_7)
act1_5_75_7 = Activation('relu')(bn1_5_75_7)
max1_5_75_7 = MaxPooling2D(pool_size,padding='same')(act1_5_75_7)
conv1_15_25_1 = Conv2D(15,(25,1),padding='same')(input_melS)
bn1_15_25_1 = BatchNormalization()(conv1_15_25_1)
act1_15_25_1 = Activation('relu')(bn1_15_25_1)
max1_15_25_1 = MaxPooling2D(pool_size,padding='same')(act1_15_25_1)
conv1_10_25_3 = Conv2D(10,(25,3),padding='same')(input_melS)
bn1_10_25_3 = BatchNormalization()(conv1_10_25_3)
act1_10_25_3 = Activation('relu')(bn1_10_25_3)
max1_10_25_3 = MaxPooling2D(pool_size,padding='same')(act1_10_25_3)
conv1_5_25_5 = Conv2D(5,(25,5),padding='same')(input_melS)
bn1_5_25_5 = BatchNormalization()(conv1_5_25_5)
act1_5_25_5 = Activation('relu')(bn1_5_25_5)
max1_5_25_5 = MaxPooling2D(pool_size,padding='same')(act1_5_25_5)
conv1_5_25_7 = Conv2D(5,(25,7),padding='same')(input_melS)
bn1_5_25_7 = BatchNormalization()(conv1_5_25_7)
act1_5_25_7 = Activation('relu')(bn1_5_25_7)
max1_5_25_7= MaxPooling2D(pool_size,padding='same')(act1_5_25_7)

max1 = Concatenate()([max1_10_96_1,max1_6_96_3,max1_3_96_5,max1_3_96_7,max1_15_75_1,max1_10_75_3,
                        max1_5_75_5,max1_5_75_7,max1_15_25_1,max1_10_25_3,max1_5_25_5,max1_5_25_7])

conv2 = Conv2D(32,(8,8),padding='same')(max1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('relu')(bn2)
max2 = MaxPooling2D(pool_size,padding='same')(act2)

flat_max_2 = Flatten()(max2)
drop = Dropout(0.5)(flat_max_2)
fc1 = Dense(100, activation='relu')(drop)
out = Dense(50, activation='sigmoid')(fc1)

model = Model(inputs = [input_melS], outputs = [out])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

filepath='C:/DT/model_save/Timbrecnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_val = np.load(path+'\\x_validation_MelSpec.npy')
y_val = np.load(path+'\\y_validation.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.aucs_all = []
        self.recalls_1 = []
        self.recalls_3 = []
        self.recalls_5 = []
        self.recalls_10 = []
        self.ndcgs_50 = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        
        recall_1 = recall(y_test,y_pred,1)
        recall_3 = recall(y_test,y_pred,3)
        recall_5 = recall(y_test,y_pred,5)
        recall_10 = recall(y_test,y_pred,10)
        self.recalls_1.append(recall_1)
        self.recalls_3.append(recall_3)
        self.recalls_5.append(recall_5)
        self.recalls_10.append(recall_10)
        
        ndcg_50 = ndcg_score(y_test,y_pred,k=50,gains="exponential")
        self.ndcgs_50.append(ndcg_50)
        
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        auc_all = roc_auc_score(y_test, y_pred,average = None )
        self.aucs_all.append(auc_all)
        
        print("Average AUC:",auc)
        print("Recalls:",recall_1,recall_3,recall_5,recall_10)
        print("NDCG:",ndcg_50)
        print("AUCs:",auc_all)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'project1/timbrecnn_aucs.pkl'),'wb'))
pickle.dump(histories.aucs_all,open(os.path.join(path,'project1/timbrecnn_aucs_all.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'project1/timbrecnn_losses.pkl'),'wb'))
pickle.dump(histories.recalls_1,open(os.path.join(path,'project1/timbrecnn_recalls_1.pkl'),'wb'))
pickle.dump(histories.recalls_3,open(os.path.join(path,'project1/timbrecnn_recalls_3.pkl'),'wb'))
pickle.dump(histories.recalls_5,open(os.path.join(path,'project1/timbrecnn_recalls_5.pkl'),'wb'))
pickle.dump(histories.recalls_10,open(os.path.join(path,'project1/timbrecnn_recalls_10.pkl'),'wb'))
pickle.dump(histories.ndcgs_50,open(os.path.join(path,'project1/timbrecnn_ndcgs_10.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 10) 970         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 96, 1360, 6)  1734        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 96, 1360, 3)  1443        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (

In [2]:
#  2 CNNs + FC + output  (MTAT)
import numpy as np 
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 30
input_shape = (96,1360,1)
model = Sequential()

model.add(Conv2D(32,(8,8),padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((4,4),padding='same'))

model.add(Conv2D(32,(8,8),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((4,4),padding='same'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='sigmoid'))

model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

filepath='C:/DT/model_save/cnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_val = np.load(path+'\\x_validation_MelSpec.npy')
y_val = np.load(path+'\\y_validation.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.aucs_all = []
        self.recalls_1 = []
        self.recalls_3 = []
        self.recalls_5 = []
        self.recalls_10 = []
        self.ndcgs_50 = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        
        recall_1 = recall(y_test,y_pred,1)
        recall_3 = recall(y_test,y_pred,3)
        recall_5 = recall(y_test,y_pred,5)
        recall_10 = recall(y_test,y_pred,10)
        self.recalls_1.append(recall_1)
        self.recalls_3.append(recall_3)
        self.recalls_5.append(recall_5)
        self.recalls_10.append(recall_10)
        
        ndcg_50 = ndcg_score(y_test,y_pred,k=50,gains="exponential")
        self.ndcgs_50.append(ndcg_50)
        
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        auc_all = roc_auc_score(y_test, y_pred,average = None )
        self.aucs_all.append(auc_all)
        
        print("Average AUC:",auc)
        print("Recalls:",recall_1,recall_3,recall_5,recall_10)
        print("NDCG:",ndcg_50)
        print("AUCs:",auc_all)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'project1/cnn_aucs.pkl'),'wb'))
pickle.dump(histories.aucs_all,open(os.path.join(path,'project1/cnn_aucs_all.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'project1/cnn_losses.pkl'),'wb'))
pickle.dump(histories.recalls_1,open(os.path.join(path,'project1/cnn_recalls_1.pkl'),'wb'))
pickle.dump(histories.recalls_3,open(os.path.join(path,'project1/cnn_recalls_3.pkl'),'wb'))
pickle.dump(histories.recalls_5,open(os.path.join(path,'project1/cnn_recalls_5.pkl'),'wb'))
pickle.dump(histories.recalls_10,open(os.path.join(path,'project1/cnn_recalls_10.pkl'),'wb'))
pickle.dump(histories.ndcgs_50,open(os.path.join(path,'project1/cnn_ndcgs_10.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 1360, 32)      2080      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 340, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 340, 32)       65568     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 340, 32)       128       
_________________________________________________________________
activation_2 (Activation)    (None, 24, 340, 32)       0         
__________

In [2]:
#  4 CNNs + 2RNN + output  (MTAT)
import numpy as np
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout,Reshape,GRU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 50
input_shape = (96,1360,1)
model = Sequential()

model.add(Conv2D(64,(3,3),padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2),padding='same'))

model.add(Conv2D(128,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((3,3),padding='same'))

model.add(Conv2D(128,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((4,4),padding='same'))

model.add(Conv2D(128,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((4,4),padding='same'))
model.add(Reshape((15, 128)))

model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dropout(0.3))
model.add(Dense(50, activation='sigmoid'))

model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

filepath='C:/DT/model_save/crnn/'+"{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor = 'loss')

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_val = np.load(path+'\\x_validation_MelSpec.npy')
y_val = np.load(path+'\\y_validation.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.aucs_all = []
        self.recalls_1 = []
        self.recalls_3 = []
        self.recalls_5 = []
        self.recalls_10 = []
        self.ndcgs_50 = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        
        recall_1 = recall(y_test,y_pred,1)
        recall_3 = recall(y_test,y_pred,3)
        recall_5 = recall(y_test,y_pred,5)
        recall_10 = recall(y_test,y_pred,10)
        self.recalls_1.append(recall_1)
        self.recalls_3.append(recall_3)
        self.recalls_5.append(recall_5)
        self.recalls_10.append(recall_10)
        
        ndcg_50 = ndcg_score(y_test,y_pred,k=50,gains="exponential")
        self.ndcgs_50.append(ndcg_50)
        
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        auc_all = roc_auc_score(y_test, y_pred,average = None )
        self.aucs_all.append(auc_all)
        
        print("Average AUC:",auc)
        print("Recalls:",recall_1,recall_3,recall_5,recall_10)
        print("NDCG:",ndcg_50)
        print("AUCs:",auc_all)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'project1/crnn_aucs.pkl'),'wb'))
pickle.dump(histories.aucs_all,open(os.path.join(path,'project1/crnn_aucs_all.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'project1/crnn_losses.pkl'),'wb'))
pickle.dump(histories.recalls_1,open(os.path.join(path,'project1/crnn_recalls_1.pkl'),'wb'))
pickle.dump(histories.recalls_3,open(os.path.join(path,'project1/crnn_recalls_3.pkl'),'wb'))
pickle.dump(histories.recalls_5,open(os.path.join(path,'project1/crnn_recalls_5.pkl'),'wb'))
pickle.dump(histories.recalls_10,open(os.path.join(path,'project1/crnn_recalls_10.pkl'),'wb'))
pickle.dump(histories.ndcgs_50,open(os.path.join(path,'project1/crnn_ndcgs_10.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 680, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 680, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 680, 128)      512       
_________________________________________________________________
activation_2 (Activation)    (None, 48, 680, 128)      0         
__________

In [1]:
# some useful functions for metrics calculation

def softmax(x):
    e_x = np.exp(x - np.max(x,1)[:,np.newaxis])
    return e_x/e_x.sum(1)[:,np.newaxis]

def recall(label_true,label_pred,k):
    '''the previous two inputs are 2 dimensional array
    '''
    idx = np.argpartition(label_pred, -k, 1)[:,-k:]
    r = label_true[np.repeat(np.arange(label_true.shape[0]),k),idx.flatten()].reshape(-1,k)
    recall = (np.sum(r,1)/np.sum(label_true,1)).mean()
    return recall

def dcg_score(y_true, y_score, k, gains="exponential"):

    order = np.argsort(y_score,axis = 1)[:,::-1][:,:k]
    
    y_t = y_true[np.repeat(np.arange(y_true.shape[0]),k),order.flatten()].reshape(-1,k)

    if gains == "exponential":
        gains = 2 ** y_t - 1
    elif gains == "linear":
        gains = y_t
    else:
        raise ValueError("Invalid gains option.")
    discounts = np.log2(np.arange(len(y_t[0,:])) + 2)[np.newaxis,:]
    dcg = np.sum(gains / discounts,1)
    return dcg

def ndcg_score(y_true, y_score, k, gains="exponential"):
   
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_score, k, gains)
    return (actual / best).mean()



#  4 CNNs + GridRNN + output  (MTAT)
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import regularizers

epochs = 50
input_shape = (96,1360,1)
input_melS = Input(input_shape)
lam = 0.0000001

conv1 = Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(lam),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(lam),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(lam),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(lam),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru1_in = Reshape((6, 128*6))(pool4)
gru1 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru1_in)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(pool4)
gru2_in = Reshape((6, 128*6))(gru2_in_perm)
gru2 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru2_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru3_in = Reshape((6, 128*6))(gru3_in_perm)
gru3 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru3_in)

gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru3_in_perm)
gru4_in = Reshape((6, 128*6))(gru4_in_perm)
gru4 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru4_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru5_in = Reshape((6, 128*6))(gru5_in_perm)
gru5 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru5_in)

gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru6_in = Reshape((6, 128*6))(gru6_in_perm)
gru6 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru6_in)

gru7_in_perm = Lambda(K.reverse,arguments={'axes':2})(gru5_in_perm)
gru7_in = Reshape((6, 128*6))(gru7_in_perm)
gru7 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru7_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1})(gru6_in_perm)
gru8_in = Reshape((6, 128*6))(gru8_in_perm)
gru8 = GRU(32,kernel_regularizer=regularizers.l2(lam))(gru8_in)

# gru1_in = Reshape((6, 128*6))(pool4)
# gru1 = GRU(32)(gru1_in)

# gru2_in_perm = K.permute_dimensions(pool4,pattern = (0,2,1,3))
# gru2_in = Reshape((6, 128*6))(gru2_in_perm)
# gru2 = GRU(32)(gru2_in)

# gru3_in_perm = K.reverse(pool4, axes = 2)
# gru3_in = Reshape((6, 128*6))(gru3_in_perm)
# gru3 = GRU(32)(gru3_in)

# gru4_in_perm = K.permute_dimensions(gru3_in_perm,pattern = (0,2,1,3))
# gru4_in = Reshape((6, 128*6))(gru4_in_perm)
# gru4 = GRU(32)(gru4_in)

# gru5_in_perm = K.reverse(pool4, axes = 1)
# gru5_in = Reshape((6, 128*6))(gru5_in_perm)
# gru5 = GRU(32)(gru5_in)

# gru6_in_perm = K.permute_dimensions(gru5_in_perm,pattern = (0,2,1,3))
# gru6_in = Reshape((6, 128*6))(gru6_in_perm)
# gru6 = GRU(32)(gru6_in)

# gru7_in_perm = K.reverse(gru5_in_perm, axes = 2)
# gru7_in = Reshape((6, 128*6))(gru7_in_perm)
# gru7 = GRU(32)(gru7_in)

# gru8_in_perm = K.reverse(gru6_in_perm, axes = 1)
# gru8_in = Reshape((6, 128*6))(gru8_in_perm)
# gru8 = GRU(32)(gru8_in)

gru = Concatenate()([gru1,gru2,gru3,gru4,gru5,gru6,gru7,gru8])
gru_dropout = Dropout(0.5)(gru)
fc = Dense(64, activation='elu',kernel_regularizer=regularizers.l2(lam))(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

filepath='C:/DT/model_save/cGrnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_val = np.load(path+'\\x_validation_MelSpec.npy')
y_val = np.load(path+'\\y_validation.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.aucs_all = []
        self.recalls_1 = []
        self.recalls_3 = []
        self.recalls_5 = []
        self.recalls_10 = []
        self.ndcgs_50 = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        
        recall_1 = recall(y_test,y_pred,1)
        recall_3 = recall(y_test,y_pred,3)
        recall_5 = recall(y_test,y_pred,5)
        recall_10 = recall(y_test,y_pred,10)
        self.recalls_1.append(recall_1)
        self.recalls_3.append(recall_3)
        self.recalls_5.append(recall_5)
        self.recalls_10.append(recall_10)
        
        ndcg_50 = ndcg_score(y_test,y_pred,k=50,gains="exponential")
        self.ndcgs_50.append(ndcg_50)
        
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        auc_all = roc_auc_score(y_test, y_pred,average = None )
        self.aucs_all.append(auc_all)
        
        print("Average AUC:",auc)
        print("Recalls:",recall_1,recall_3,recall_5,recall_10)
        print("NDCG:",ndcg_50)
        print("AUCs:",auc_all)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'project1/cGrnn_aucs.pkl'),'wb'))
pickle.dump(histories.aucs_all,open(os.path.join(path,'project1/cGrnn_aucs_all.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'project1/cGrnn_losses.pkl'),'wb'))
pickle.dump(histories.recalls_1,open(os.path.join(path,'project1/cGrnn_recalls_1.pkl'),'wb'))
pickle.dump(histories.recalls_3,open(os.path.join(path,'project1/cGrnn_recalls_3.pkl'),'wb'))
pickle.dump(histories.recalls_5,open(os.path.join(path,'project1/cGrnn_recalls_5.pkl'),'wb'))
pickle.dump(histories.recalls_10,open(os.path.join(path,'project1/cGrnn_recalls_10.pkl'),'wb'))
pickle.dump(histories.ndcgs_50,open(os.path.join(path,'project1/cGrnn_ndcgs_10.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
# Merge into big files  (FMA)
import numpy as np
import pickle
import os

x = np.zeros([19909,96,1360,1])
file_list = pickle.load(open('C:/DT/fma_medium/train_list.pkl','rb'))
for i,file in enumerate(file_list):
    x[i] = np.load(os.path.join('C:/DT/fma_medium/train',file))[...,np.newaxis]
np.save('C:/DT/fma_medium/x_train.npy',x)
print(i)
x = np.zeros([2504,96,1360,1])
file_list = pickle.load(open('C:/DT/fma_medium/val_list.pkl','rb'))
for i,file in enumerate(file_list):
    x[i] = np.load(os.path.join('C:/DT/fma_medium/val',file))[...,np.newaxis]
np.save('C:/DT/fma_medium/x_val.npy',x)
print(i)
x = np.zeros([2572,96,1360,1])
file_list = pickle.load(open('C:/DT/fma_medium/test_list.pkl','rb'))
for i,file in enumerate(file_list):
    x[i] = np.load(os.path.join('C:/DT/fma_medium/test',file))[...,np.newaxis]
np.save('C:/DT/fma_medium/x_test.npy',x)
print(i)

19908
2503
2571


In [1]:
#  FCN-4 + output  (FMA)
import numpy as np 
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 30
input_shape = (96,1360,1)
model = Sequential()

model.add(Conv2D(128,(3,3),padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,4),padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(384,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,5),strides=(2,4),padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(768,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,8),strides=(2,4),padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(2048,(3,3),strides=(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,8),strides=(4,8),padding='same'))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(16, activation='softmax'))

model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

filepath='C:/DT/model_save/fma/fcn/'+"{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
y_train_idx = np.argmax(y_train,axis = -1)
x_test = np.load(path+'\\x_test.npy')
y_test = np.load(path+'\\test_y.npy')
y_test_idx = np.argmax(y_test,axis = -1)

AUC_max = 0
Recall_weighted_max = 0
Recall_macro_max = 0
Precision_weighted_max = 0
Precision_macro_max = 0
F1_score_weighted_max = 0
F1_score_macro_max = 0
ACC_max = 0

from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        # all evaluation metrics include train, val, test except accs
        self.aucs = [] # weighted AUC
        self.recalls = [] # 16 recalls on each class and 1 mean recall
        self.precisions = [] # 16 precisions on each class and 1 mean precision
        self.f1s = [] # 16 f1s on each class and 1 mean f1 
        self.accs = [] # mean accuracy without take imblance into account

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        global AUC_max, Recall_weighted_max, Recall_macro_max, Precision_weighted_max, Precision_macro_max, \
                F1_score_weighted_max, F1_score_macro_max, ACC_max
        y_pred_test = self.model.predict(x_test)
        y_pred_test_idx = np.argmax(y_pred_test,axis = -1)
                
        auc = []
        auc_test = roc_auc_score(y_test, y_pred_test,'weighted')
        if auc_test > AUC_max:
            AUC_max = auc_test
        auc.append(auc_test)
        self.aucs.append(auc)
        print("Weighted AUCs:",auc)
        
        recall = []
        recall_all = recall_score(y_test_idx,y_pred_test_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='weighted')
        if recall_mean > Recall_weighted_max:
            Recall_weighted_max = recall_mean
        recall.append(recall_mean)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='macro')
        if recall_mean > Recall_macro_max:
            Recall_macro_max = recall_mean
        recall.append(recall_mean)
        self.recalls.append(recall)
        print("weighted average Recalls:",recall[1])
        print("unweighted average Recalls (macro):",recall[2])
        
        precision = []
        precision_all = precision_score(y_test_idx,y_pred_test_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='weighted')
        if precision_mean > Precision_weighted_max:
            Precision_weighted_max = precision_mean
        precision.append(precision_mean)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='macro')
        if precision_mean > Precision_macro_max:
            Precision_macro_max = precision_mean
        precision.append(precision_mean)
        self.precisions.append(precision)
        print("weighted average Precisions:",precision[1])
        print("unweighted average Precisions(macro):",precision[2])
        
        f1 = []
        f1_all = f1_score(y_test_idx,y_pred_test_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='weighted')
        if f1_mean > F1_score_weighted_max:
            F1_score_weighted_max = f1_mean
        f1.append(f1_mean)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='macro')
        if f1_mean > F1_score_macro_max:
            F1_score_macro_max = f1_mean
        f1.append(f1_mean)
        self.f1s.append(f1)
        print("weighted average F1s:",f1[1])
        print("unweighted average F1s(macro):",f1[2])
        
        loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
        if acc > ACC_max:
            ACC_max = acc
        self.accs.append(acc)
        print("Test Categorical Accuracy:",acc)              
        return

histories = Histories()
history = model.fit(x_train, y_train, batch_size = 16, epochs = epochs, callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'metrics_save/fcn_aucs.pkl'),'wb'))
pickle.dump(histories.recalls,open(os.path.join(path,'metrics_save/fcn_recalls.pkl'),'wb'))
pickle.dump(histories.precisions,open(os.path.join(path,'metrics_save/fcn_precisions.pkl'),'wb'))
pickle.dump(histories.f1s,open(os.path.join(path,'metrics_save/fcn_f1s.pkl'),'wb'))
pickle.dump(histories.accs,open(os.path.join(path,'metrics_save/fcn_accs.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'metrics_save/fcn_history.pkl'),'wb'))
print('AUC_max:',AUC_max,'Recall_weighted_max:',Recall_weighted_max,'Recall_macro_max:',Recall_macro_max,
     'Precision_weighted_max:',Precision_weighted_max,'Precision_macro_max:',Precision_macro_max,
     'F1_score_weighted_max:',F1_score_weighted_max,'F1_score_macro_max:',F1_score_macro_max,
     'ACC_max:',ACC_max)

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 1360, 128)     1280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 128)     512       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 128)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 340, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 384)      442752    
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 384)      1536      
__________

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Test Categorical Accuracy: 0.5451010886469674
Epoch 2/30
19909/19909 [==============================] - 930s 47ms/step - loss: 2.0545 - categorical_accuracy: 0.4881
Weighted AUCs: [0.8487485560804802]
weighted average Recalls: 0.4265163297045101
unweighted average Recalls (macro): 0.2026837418116068
weighted average Precisions: 0.4652523709554144
unweighted average Precisions(macro): 0.24223745810785385
weighted average F1s: 0.3352545726171395
unweighted average F1s(macro): 0.150444676344648
Test Categorical Accuracy: 0.4265163297972078
Epoch 3/30
19909/19909 [==============================] - 930s 47ms/step - loss: 1.7618 - categorical_accuracy: 0.5334
Weighted AUCs: [0.8670945997416363]
weighted average Recalls: 0.48833592534992226
unweighted average Recalls (macro): 0.24411316931321492
weighted average Precisions: 0.5327589232900457
unweighted average Precisions(macro): 0.3139589867754276
weighted average F1s: 0.45414189731130256
unweighted average F1s(macro): 0.2424077975842247
Tes

In [1]:
#  Timbre CNN  (FMA)
import numpy as np 
from keras.layers import Input,Conv2D,BatchNormalization,Activation,MaxPooling2D,Concatenate,Flatten,Dropout,Dense
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

epochs = 30
pool_size = (4,4)
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1_10_96_1 = Conv2D(10,(96,1),padding='same')(input_melS)
bn1_10_96_1 = BatchNormalization()(conv1_10_96_1)
act1_10_96_1 = Activation('relu')(bn1_10_96_1)
max1_10_96_1 = MaxPooling2D(pool_size,padding='same')(act1_10_96_1)
conv1_6_96_3 = Conv2D(6,(96,3),padding='same')(input_melS)
bn1_6_96_3 = BatchNormalization()(conv1_6_96_3)
act1_6_96_3 = Activation('relu')(bn1_6_96_3)
max1_6_96_3 = MaxPooling2D(pool_size,padding='same')(act1_6_96_3)
conv1_3_96_5 = Conv2D(3,(96,5),padding='same')(input_melS)
bn1_3_96_5 = BatchNormalization()(conv1_3_96_5)
act1_3_96_5 = Activation('relu')(bn1_3_96_5)
max1_3_96_5 = MaxPooling2D(pool_size,padding='same')(act1_3_96_5)
conv1_3_96_7 = Conv2D(3,(96,7),padding='same')(input_melS)
bn1_3_96_7 = BatchNormalization()(conv1_3_96_7)
act1_3_96_7 = Activation('relu')(bn1_3_96_7)
max1_3_96_7 = MaxPooling2D(pool_size,padding='same')(act1_3_96_7)
conv1_15_75_1 = Conv2D(15,(75,1),padding='same')(input_melS)
bn1_15_75_1 = BatchNormalization()(conv1_15_75_1)
act1_15_75_1 = Activation('relu')(bn1_15_75_1)
max1_15_75_1 = MaxPooling2D(pool_size,padding='same')(act1_15_75_1)
conv1_10_75_3 = Conv2D(6,(75,3),padding='same')(input_melS)
bn1_10_75_3 = BatchNormalization()(conv1_10_75_3)
act1_10_75_3 = Activation('relu')(bn1_10_75_3)
max1_10_75_3 = MaxPooling2D(pool_size,padding='same')(act1_10_75_3)
conv1_5_75_5 = Conv2D(3,(75,5),padding='same')(input_melS)
bn1_5_75_5 = BatchNormalization()(conv1_5_75_5)
act1_5_75_5 = Activation('relu')(bn1_5_75_5)
max1_5_75_5 = MaxPooling2D(pool_size,padding='same')(act1_5_75_5)
conv1_5_75_7 = Conv2D(3,(75,7),padding='same')(input_melS)
bn1_5_75_7 = BatchNormalization()(conv1_5_75_7)
act1_5_75_7 = Activation('relu')(bn1_5_75_7)
max1_5_75_7 = MaxPooling2D(pool_size,padding='same')(act1_5_75_7)
conv1_15_25_1 = Conv2D(15,(25,1),padding='same')(input_melS)
bn1_15_25_1 = BatchNormalization()(conv1_15_25_1)
act1_15_25_1 = Activation('relu')(bn1_15_25_1)
max1_15_25_1 = MaxPooling2D(pool_size,padding='same')(act1_15_25_1)
conv1_10_25_3 = Conv2D(10,(25,3),padding='same')(input_melS)
bn1_10_25_3 = BatchNormalization()(conv1_10_25_3)
act1_10_25_3 = Activation('relu')(bn1_10_25_3)
max1_10_25_3 = MaxPooling2D(pool_size,padding='same')(act1_10_25_3)
conv1_5_25_5 = Conv2D(5,(25,5),padding='same')(input_melS)
bn1_5_25_5 = BatchNormalization()(conv1_5_25_5)
act1_5_25_5 = Activation('relu')(bn1_5_25_5)
max1_5_25_5 = MaxPooling2D(pool_size,padding='same')(act1_5_25_5)
conv1_5_25_7 = Conv2D(5,(25,7),padding='same')(input_melS)
bn1_5_25_7 = BatchNormalization()(conv1_5_25_7)
act1_5_25_7 = Activation('relu')(bn1_5_25_7)
max1_5_25_7= MaxPooling2D(pool_size,padding='same')(act1_5_25_7)

max1 = Concatenate()([max1_10_96_1,max1_6_96_3,max1_3_96_5,max1_3_96_7,max1_15_75_1,max1_10_75_3,
                        max1_5_75_5,max1_5_75_7,max1_15_25_1,max1_10_25_3,max1_5_25_5,max1_5_25_7])

conv2 = Conv2D(32,(8,8),padding='same')(max1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('relu')(bn2)
max2 = MaxPooling2D(pool_size,padding='same')(act2)

flat_max_2 = Flatten()(max2)
drop = Dropout(0.5)(flat_max_2)
fc1 = Dense(100, activation='relu')(drop)
out = Dense(16, activation='softmax')(fc1)

model = Model(inputs = [input_melS], outputs = [out])

model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

filepath='C:/DT/model_save/fma/timbrecnn/'+"{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
y_train_idx = np.argmax(y_train,axis = -1)
x_test = np.load(path+'\\x_test.npy')
y_test = np.load(path+'\\test_y.npy')
y_test_idx = np.argmax(y_test,axis = -1)

AUC_max = 0
Recall_weighted_max = 0
Recall_macro_max = 0
Precision_weighted_max = 0
Precision_macro_max = 0
F1_score_weighted_max = 0
F1_score_macro_max = 0
ACC_max = 0

from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        # all evaluation metrics include train, val, test except accs
        self.aucs = [] # weighted AUC
        self.recalls = [] # 16 recalls on each class and 1 mean recall
        self.precisions = [] # 16 precisions on each class and 1 mean precision
        self.f1s = [] # 16 f1s on each class and 1 mean f1 
        self.accs = [] # mean accuracy without take imblance into account

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        global AUC_max, Recall_weighted_max, Recall_macro_max, Precision_weighted_max, Precision_macro_max, \
                F1_score_weighted_max, F1_score_macro_max, ACC_max
        y_pred_test = self.model.predict(x_test)
        y_pred_test_idx = np.argmax(y_pred_test,axis = -1)
                
        auc = [] 
        auc_test = roc_auc_score(y_test, y_pred_test,'weighted')
        if auc_test > AUC_max:
            AUC_max = auc_test
        auc.append(auc_test)
        self.aucs.append(auc)
        print("Weighted AUCs:",auc)
        
        recall = []
        recall_all = recall_score(y_test_idx,y_pred_test_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='weighted')
        if recall_mean > Recall_weighted_max:
            Recall_weighted_max = recall_mean
        recall.append(recall_mean)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='macro')
        if recall_mean > Recall_macro_max:
            Recall_macro_max = recall_mean
        recall.append(recall_mean)
        self.recalls.append(recall)
        print("weighted average Recalls:",recall[1])
        print("unweighted average Recalls (macro):",recall[2])
        
        precision = []
        precision_all = precision_score(y_test_idx,y_pred_test_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='weighted')
        if precision_mean > Precision_weighted_max:
            Precision_weighted_max = precision_mean
        precision.append(precision_mean)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='macro')
        if precision_mean > Precision_macro_max:
            Precision_macro_max = precision_mean
        precision.append(precision_mean)
        self.precisions.append(precision)
        print("weighted average Precisions:",precision[1])
        print("unweighted average Precisions(macro):",precision[2])
        
        f1 = []
        f1_all = f1_score(y_test_idx,y_pred_test_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='weighted')
        if f1_mean > F1_score_weighted_max:
            F1_score_weighted_max = f1_mean
        f1.append(f1_mean)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='macro')
        if f1_mean > F1_score_macro_max:
            F1_score_macro_max = f1_mean
        f1.append(f1_mean)
        self.f1s.append(f1)
        print("weighted average F1s:",f1[1])
        print("unweighted average F1s(macro):",f1[2])
        
        loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
        if acc > ACC_max:
            ACC_max = acc
        self.accs.append(acc)
        print("Test Categorical Accuracy:",acc)
                
        return

histories = Histories()
history = model.fit(x_train, y_train, batch_size = 16, epochs = epochs, callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'metrics_save/timbrecnn_aucs.pkl'),'wb'))
pickle.dump(histories.recalls,open(os.path.join(path,'metrics_save/timbrecnn_recalls.pkl'),'wb'))
pickle.dump(histories.precisions,open(os.path.join(path,'metrics_save/timbrecnn_precisions.pkl'),'wb'))
pickle.dump(histories.f1s,open(os.path.join(path,'metrics_save/timbrecnn_f1s.pkl'),'wb'))
pickle.dump(histories.accs,open(os.path.join(path,'metrics_save/timbrecnn_accs.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'metrics_save/timbrecnn_history.pkl'),'wb'))
print('AUC_max:',AUC_max,'Recall_weighted_max:',Recall_weighted_max,'Recall_macro_max:',Recall_macro_max,
     'Precision_weighted_max:',Precision_weighted_max,'Precision_macro_max:',Precision_macro_max,
     'F1_score_weighted_max:',F1_score_weighted_max,'F1_score_macro_max:',F1_score_macro_max,
     'ACC_max:',ACC_max)

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 10) 970         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 96, 1360, 6)  1734        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 96, 1360, 3)  1443        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Test Categorical Accuracy: 0.44673405909797825
Epoch 2/30
19909/19909 [==============================] - 1284s 64ms/step - loss: 1.6756 - categorical_accuracy: 0.4644
Weighted AUCs: [0.8179792056822771]
weighted average Recalls: 0.45295489891135304
unweighted average Recalls (macro): 0.12753095419433472
weighted average Precisions: 0.314055836590573
unweighted average Precisions(macro): 0.09402403618436266
weighted average F1s: 0.3592544642441051
unweighted average F1s(macro): 0.10081937767546441
Test Categorical Accuracy: 0.45295489891135304
Epoch 3/30
19909/19909 [==============================] - 1281s 64ms/step - loss: 1.5601 - categorical_accuracy: 0.5029
Weighted AUCs: [0.8410250616251833]
weighted average Recalls: 0.5217729393468118
unweighted average Recalls (macro): 0.2519197826708527
weighted average Precisions: 0.44058402750578324
unweighted average Precisions(macro): 0.2138336431736346
weighted average F1s: 0.46298426581505986
unweighted average F1s(macro): 0.21856816581025

In [1]:
#  4 CNNs + GridRNN + output  (FMA)
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 40
input_shape = (96,1360,1)
input_melS = Input(input_shape,name='main_input')

conv1 = Conv2D(64,(3,3),padding='same',name='conv_1')(input_melS)
bn1 = BatchNormalization(name='bn_1')(conv1)
act1 = Activation('elu',name='act_1')(bn1)
pool1 = MaxPooling2D((2,4),padding='same',name='pool_1')(act1)

conv2 = Conv2D(128,(3,3),padding='same',name='conv_2')(pool1)
bn2 = BatchNormalization(name='bn_2')(conv2)
act2 = Activation('elu',name='act_2')(bn2)
pool2 = MaxPooling2D((2,4),padding='same',name='pool_2')(act2)

conv3 = Conv2D(128,(3,3),padding='same',name='conv_3')(pool2)
bn3 = BatchNormalization(name='bn_3')(conv3)
act3 = Activation('elu',name='act_3')(bn3)
pool3 = MaxPooling2D((2,4),padding='same',name='pool_3')(act3)

conv4 = Conv2D(128,(3,3),padding='same',name='conv_4')(pool3)
bn4 = BatchNormalization(name='bn_4')(conv4)
act4 = Activation('elu',name='act_4')(bn4)
pool4 = MaxPooling2D((2,4),padding='same',name='pool_4')(act4)

gru1_in = Reshape((6, 128*6),name='gru1_in')(pool4)
gru1 = GRU(32,name='gru1')(gru1_in)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru2_in_perm')(pool4)
gru2_in = Reshape((6, 128*6),name='gru2_in')(gru2_in_perm)
gru2 = GRU(32,name='gru2')(gru2_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2},name='gru3_in_perm')(pool4)
gru3_in = Reshape((6, 128*6),name='gru3_in')(gru3_in_perm)
gru3 = GRU(32,name='gru3')(gru3_in)

gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru4_in_perm')(gru3_in_perm)
gru4_in = Reshape((6, 128*6),name='gru4_in')(gru4_in_perm)
gru4 = GRU(32,name='gru4')(gru4_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1},name='gru5_in_perm')(pool4)
gru5_in = Reshape((6, 128*6),name='gru5_in')(gru5_in_perm)
gru5 = GRU(32,name='gru5')(gru5_in)

gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru6_in_perm')(gru5_in_perm)
gru6_in = Reshape((6, 128*6),name='gru6_in')(gru6_in_perm)
gru6 = GRU(32,name='gru6')(gru6_in)

gru7_in_perm = Lambda(K.reverse,arguments={'axes':2},name='gru7_in_perm')(gru5_in_perm)
gru7_in = Reshape((6, 128*6),name='gru7_in')(gru7_in_perm)
gru7 = GRU(32,name='gru7')(gru7_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1},name='gru8_in_perm')(gru6_in_perm)
gru8_in = Reshape((6, 128*6),name='gru8_in')(gru8_in_perm)
gru8 = GRU(32,name='gru8')(gru8_in)

gru = Concatenate(name='gru_conc')([gru1,gru2,gru3,gru4,gru5,gru6,gru7,gru8])
gru_dropout = Dropout(0.5,name='gru_conc_dropout')(gru)
fc = Dense(64, activation='elu',name='fc')(gru_dropout)
pred = Dense(16, activation='softmax',name='pred')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

filepath='C:/DT/model_save/fma/cGrnn/'+"{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
y_train_idx = np.argmax(y_train,axis = -1)
x_test = np.load(path+'\\x_test.npy')
y_test = np.load(path+'\\test_y.npy')
y_test_idx = np.argmax(y_test,axis = -1)

AUC_max = 0
Recall_weighted_max = 0
Recall_macro_max = 0
Precision_weighted_max = 0
Precision_macro_max = 0
F1_score_weighted_max = 0
F1_score_macro_max = 0
ACC_max = 0

from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        # all evaluation metrics include train, val, test except accs
        self.aucs = [] # weighted AUC
        self.recalls = [] # 16 recalls on each class and 1 mean recall
        self.precisions = [] # 16 precisions on each class and 1 mean precision
        self.f1s = [] # 16 f1s on each class and 1 mean f1 
        self.accs = [] # mean accuracy without take imblance into account

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        global AUC_max, Recall_weighted_max, Recall_macro_max, Precision_weighted_max, Precision_macro_max, \
                F1_score_weighted_max, F1_score_macro_max, ACC_max
        y_pred_test = self.model.predict(x_test)
        y_pred_test_idx = np.argmax(y_pred_test,axis = -1)
                
        auc = [] 
        auc_test = roc_auc_score(y_test, y_pred_test,'weighted')
        if auc_test > AUC_max:
            AUC_max = auc_test
        auc.append(auc_test)
        self.aucs.append(auc)
        print("Weighted AUCs:",auc)
        
        recall = []
        recall_all = recall_score(y_test_idx,y_pred_test_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='weighted')
        if recall_mean > Recall_weighted_max:
            Recall_weighted_max = recall_mean
        recall.append(recall_mean)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='macro')
        if recall_mean > Recall_macro_max:
            Recall_macro_max = recall_mean
        recall.append(recall_mean)
        self.recalls.append(recall)
        print("weighted average Recalls:",recall[1])
        print("unweighted average Recalls (macro):",recall[2])
        
        precision = []
        precision_all = precision_score(y_test_idx,y_pred_test_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='weighted')
        if precision_mean > Precision_weighted_max:
            Precision_weighted_max = precision_mean
        precision.append(precision_mean)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='macro')
        if precision_mean > Precision_macro_max:
            Precision_macro_max = precision_mean
        precision.append(precision_mean)
        self.precisions.append(precision)
        print("weighted average Precisions:",precision[1])
        print("unweighted average Precisions(macro):",precision[2])
        
        f1 = []
        f1_all = f1_score(y_test_idx,y_pred_test_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='weighted')
        if f1_mean > F1_score_weighted_max:
            F1_score_weighted_max = f1_mean
        f1.append(f1_mean)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='macro')
        if f1_mean > F1_score_macro_max:
            F1_score_macro_max = f1_mean
        f1.append(f1_mean)
        self.f1s.append(f1)
        print("weighted average F1s:",f1[1])
        print("unweighted average F1s(macro):",f1[2])
        
        loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
        if acc > ACC_max:
            ACC_max = acc
        self.accs.append(acc)
        print("Test Categorical Accuracy:",acc)
                
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'metrics_save/cGrnn_aucs_1.pkl'),'wb'))
pickle.dump(histories.recalls,open(os.path.join(path,'metrics_save/cGrnn_recalls_1.pkl'),'wb'))
pickle.dump(histories.precisions,open(os.path.join(path,'metrics_save/cGrnn_precisions_1.pkl'),'wb'))
pickle.dump(histories.f1s,open(os.path.join(path,'metrics_save/cGrnn_f1s_1.pkl'),'wb'))
pickle.dump(histories.accs,open(os.path.join(path,'metrics_save/cGrnn_accs_1.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'metrics_save/cGrnn_history_1.pkl'),'wb'))
print('AUC_max:',AUC_max,'Recall_weighted_max:',Recall_weighted_max,'Recall_macro_max:',Recall_macro_max,
     'Precision_weighted_max:',Precision_weighted_max,'Precision_macro_max:',Precision_macro_max,
     'F1_score_weighted_max:',F1_score_weighted_max,'F1_score_macro_max:',F1_score_macro_max,
     'ACC_max:',ACC_max)

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 1360, 64) 640         main_input[0][0]                 
__________________________________________________________________________________________________
bn_1 (BatchNormalization)       (None, 96, 1360, 64) 256         conv_1[0][0]                     
__________________________________________________________________________________________________
act_1 (Activation)              (None, 96, 1360, 64) 0           bn_1[0][0]                       
__________________________________________________________________________________________________
pool_1 (Ma

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Test Categorical Accuracy: 0.3856920684292379
Epoch 2/40
19909/19909 [==============================] - 393s 20ms/step - loss: 1.3157 - categorical_accuracy: 0.5953
Weighted AUCs: [0.7610444715035509]
weighted average Recalls: 0.19828926905132194
unweighted average Recalls (macro): 0.1826501857934314
weighted average Precisions: 0.47376740908386067
unweighted average Precisions(macro): 0.26257289043588417
weighted average F1s: 0.15380298047914617
unweighted average F1s(macro): 0.13610893686925687
Test Categorical Accuracy: 0.19828926914401968
Epoch 3/40
19909/19909 [==============================] - 393s 20ms/step - loss: 1.2489 - categorical_accuracy: 0.6149
Weighted AUCs: [0.8976067434023407]
weighted average Recalls: 0.5991446345256609
unweighted average Recalls (macro): 0.3675070066628865
weighted average Precisions: 0.56930891624832
unweighted average Precisions(macro): 0.35018987545639385
weighted average F1s: 0.5638099356465712
unweighted average F1s(macro): 0.33539106533481927


In [1]:
# #  2 CNNs + FC + output  (FMA)
# import numpy as np 
# from keras.callbacks import Callback
# from keras.models import Sequential
# from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint

# epochs = 30
# input_shape = (96,1360,1)
# model = Sequential()

# model.add(Conv2D(32,(8,8),padding='same',input_shape=input_shape))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D((4,4),padding='same'))

# model.add(Conv2D(32,(8,8),padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D((4,4),padding='same'))

# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(16, activation='sigmoid'))

# model.summary()
# model.compile(loss = 'binary_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

# filepath='C:/DT/model_save/fma/cnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath,save_best_only = 'True',mode = 'min')

# path = 'C:/DT/fma_medium'
# x_train = np.load(path+'\\x_train.npy')
# y_train = np.load(path+'\\train_y.npy')
# x_val = np.load(path+'\\x_val.npy')
# y_val = np.load(path+'\\val_y.npy')
# x_test = np.load(path+'\\x_test.npy')
# y_test = np.load(path+'\\test_y.npy')

# from sklearn.metrics import roc_auc_score
# class Histories(Callback):
#     def on_train_begin(self, logs={}):
#         self.aucs = []
#         self.accs = []

#     def on_train_end(self, logs={}):
#         return

#     def on_epoch_begin(self, epoch, logs={}):
#         return

#     def on_epoch_end(self, epoch, logs={}):
#         auc = []
#         y_pred = self.model.predict(x_train)
#         auc_train = roc_auc_score(y_train, y_pred)
#         auc.append(auc_train)
#         y_pred = self.model.predict(x_val)
#         auc_val = roc_auc_score(y_val, y_pred)
#         auc.append(auc_val)
#         y_pred = self.model.predict(x_test)
#         auc_test = roc_auc_score(y_test, y_pred)
#         auc.append(auc_test)
#         self.aucs.append(auc)
#         print("AUCs:",auc) 
        
#         loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
#         self.accs.append(acc)
#         print("Test Acc:",acc)
        
        
#         return

#     def on_batch_begin(self, batch, logs={}):
#         return

#     def on_batch_end(self, batch, logs={}):
#         return

# histories = Histories()
# history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

# import pickle
# import os
# pickle.dump(histories.accs,open(os.path.join(path,'cnn_accs_test.pkl'),'wb'))
# pickle.dump(history.history,open(os.path.join(path,'cnn_accLoss_trainVal.pkl'),'wb'))


#  2 CNNs + FC + output  (FMA)
import numpy as np 
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 30
input_shape = (96,1360,1)
model = Sequential()

model.add(Conv2D(32,(8,8),padding='same',input_shape=input_shape,name='conv1'))
model.add(BatchNormalization(name='bn1'))
model.add(Activation('relu',name='act1'))
model.add(MaxPooling2D((4,4),padding='same',name='pool1'))

model.add(Conv2D(32,(8,8),padding='same',name='conv2'))
model.add(BatchNormalization(name='bn2'))
model.add(Activation('relu',name='act2'))
model.add(MaxPooling2D((4,4),padding='same',name='pool2'))

model.add(Flatten(name='flatten'))
model.add(Dropout(0.5,name='dropout'))
model.add(Dense(100, activation='relu',name='ds1'))
model.add(Dense(16, activation='softmax',name='ds2'))

model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

filepath='C:/DT/model_save/fma/cnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
y_train_idx = np.argmax(y_train,axis = -1)
x_val = np.load(path+'\\x_val.npy')
y_val = np.load(path+'\\val_y.npy')
y_val_idx = np.argmax(y_val,axis = -1)
x_test = np.load(path+'\\x_test.npy')
y_test = np.load(path+'\\test_y.npy')
y_test_idx = np.argmax(y_test,axis = -1)

from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        # all evaluation metrics include train, val, test except accs
        self.aucs = [] # weighted AUC
        self.recalls = [] # 16 recalls on each class and 1 mean recall
        self.precisions = [] # 16 precisions on each class and 1 mean precision
        self.f1s = [] # 16 f1s on each class and 1 mean f1 
        self.accs = [] # mean accuracy without take imblance into account

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = self.model.predict(x_train)
        y_pred_val = self.model.predict(x_val)
        y_pred_test = self.model.predict(x_test)
        
        y_pred_train_idx = np.argmax(y_pred_train,axis = -1)
        y_pred_val_idx = np.argmax(y_pred_val,axis = -1)
        y_pred_test_idx = np.argmax(y_pred_test,axis = -1)
                
        auc = [] #[train,val,test]
        auc_train = roc_auc_score(y_train, y_pred_train,'weighted')
        auc.append(auc_train)
        auc_val = roc_auc_score(y_val, y_pred_val,'weighted')
        auc.append(auc_val)
        auc_test = roc_auc_score(y_test, y_pred_test,'weighted')
        auc.append(auc_test)
        self.aucs.append(auc)
        print("Weighted AUCs:",auc)
        
        recall = []
        recall_all = recall_score(y_train_idx,y_pred_train_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_train_idx,y_pred_train_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_train_idx,y_pred_train_idx,average='macro')
        recall.append(recall_mean)
        recall_all = recall_score(y_val_idx,y_pred_val_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_val_idx,y_pred_val_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_val_idx,y_pred_val_idx,average='macro')
        recall.append(recall_mean)
        recall_all = recall_score(y_test_idx,y_pred_test_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='macro')
        recall.append(recall_mean)
        self.recalls.append(recall)
        print("weighted average Recalls:",recall[1],recall[4],recall[7])
        print("unweighted average Recalls (macro):",recall[2],recall[5],recall[8])
        
        precision = []
        precision_all = precision_score(y_train_idx,y_pred_train_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_train_idx,y_pred_train_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_train_idx,y_pred_train_idx,average='macro')
        precision.append(precision_mean)
        precision_all = precision_score(y_val_idx,y_pred_val_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_val_idx,y_pred_val_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_val_idx,y_pred_val_idx,average='macro')
        precision.append(precision_mean)
        precision_all = precision_score(y_test_idx,y_pred_test_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='macro')
        precision.append(precision_mean)
        self.precisions.append(precision)
        print("weighted average Precisions:",precision[1],precision[4],precision[7])
        print("unweighted average Precisions(macro):",precision[2],precision[5],precision[8])
        
        f1 = []
        f1_all = f1_score(y_train_idx,y_pred_train_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='macro')
        f1.append(f1_mean)
        f1_all = f1_score(y_val_idx,y_pred_val_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_val_idx,y_pred_val_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_val_idx,y_pred_val_idx,average='macro')
        f1.append(f1_mean)
        f1_all = f1_score(y_test_idx,y_pred_test_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='macro')
        f1.append(f1_mean)
        self.f1s.append(f1)
        print("weighted average F1s:",f1[1],f1[4],f1[7])
        print("unweighted average F1s(macro):",f1[2],f1[5],f1[8])
        
        loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
        self.accs.append(acc)
        print("Test Categorical Accuracy:",acc)
                
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'metrics_save/cnn_aucs.pkl'),'wb'))
pickle.dump(histories.recalls,open(os.path.join(path,'metrics_save/cnn_recalls.pkl'),'wb'))
pickle.dump(histories.precisions,open(os.path.join(path,'metrics_save/cnn_precisions.pkl'),'wb'))
pickle.dump(histories.f1s,open(os.path.join(path,'metrics_save/cnn_f1s.pkl'),'wb'))
pickle.dump(histories.accs,open(os.path.join(path,'metrics_save/cnn_accs.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'metrics_save/cnn_history.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 96, 1360, 32)      2080      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 96, 1360, 32)      128       
_________________________________________________________________
act1 (Activation)            (None, 96, 1360, 32)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 24, 340, 32)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 24, 340, 32)       65568     
_________________________________________________________________
bn2 (BatchNormalization)     (None, 24, 340, 32)       128       
_________________________________________________________________
act2 (Activation)            (None, 24, 340, 32)       0         
__________

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Test Categorical Accuracy: 0.3386469672942421
Epoch 2/30
19909/19909 [==============================] - 184s 9ms/step - loss: 1.5120 - categorical_accuracy: 0.5239 - val_loss: 1.4699 - val_categorical_accuracy: 0.5551
Weighted AUCs: [0.8609552668221925, 0.8686564279469003, 0.8591723372193198]
weighted average Recalls: 0.5344818926113818 0.555111821086262 0.5279937791601866
unweighted average Recalls: 0.23372758033253554 0.23372758033253554 0.23372758033253554
weighted average Precisions: 0.46536337748911244 0.4702855769069621 0.44067124686681375
unweighted average Precisions: 0.2655367255563348 0.2655367255563348 0.2655367255563348
weighted average F1s: 0.462912109103022 0.4805066983318777 0.4462165282347083
unweighted average F1s: 0.21244256659358784 0.21244256659358784 0.21244256659358784
Test Categorical Accuracy: 0.5279937791833611
Epoch 3/30
19909/19909 [==============================] - 184s 9ms/step - loss: 1.3958 - categorical_accuracy: 0.5703 - val_loss: 1.3856 - val_categoric

In [2]:
#  4 CNNs + 2RNN + output  (FMA)
import numpy as np
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Flatten,Dense,Dropout,Reshape,GRU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 30
input_shape = (96,1360,1)
model = Sequential()

model.add(Conv2D(64,(3,3),padding='same',input_shape=input_shape,name='conv1'))
model.add(BatchNormalization(name='bn1'))
model.add(Activation('relu',name='act1'))
model.add(MaxPooling2D((2,2),padding='same',name='pool1'))

model.add(Conv2D(128,(3,3),padding='same',name='conv2'))
model.add(BatchNormalization(name='bn2'))
model.add(Activation('relu',name='act2'))
model.add(MaxPooling2D((3,3),padding='same',name='pool2'))

model.add(Conv2D(128,(3,3),padding='same',name='conv3'))
model.add(BatchNormalization(name='bn3'))
model.add(Activation('relu',name='act3'))
model.add(MaxPooling2D((4,4),padding='same',name='pool3'))

model.add(Conv2D(128,(3,3),padding='same',name='conv4'))
model.add(BatchNormalization(name='bn4'))
model.add(Activation('relu',name='act4'))
model.add(MaxPooling2D((4,4),padding='same',name='pool4'))
model.add(Reshape((15, 128),name='rs'))

model.add(GRU(32,return_sequences=True,name='gru1'))
model.add(GRU(32,name='gru2'))
model.add(Dropout(0.3,name='dropout'))
model.add(Dense(16, activation='softmax',name='ds'))

model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

filepath='C:/DT/model_save/fma/crnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
y_train_idx = np.argmax(y_train,axis = -1)
x_val = np.load(path+'\\x_val.npy')
y_val = np.load(path+'\\val_y.npy')
y_val_idx = np.argmax(y_val,axis = -1)
x_test = np.load(path+'\\x_test.npy')
y_test = np.load(path+'\\test_y.npy')
y_test_idx = np.argmax(y_test,axis = -1)

from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        # all evaluation metrics include train, val, test except accs
        self.aucs = [] # weighted AUC
        self.recalls = [] # 16 recalls on each class and 1 mean recall
        self.precisions = [] # 16 precisions on each class and 1 mean precision
        self.f1s = [] # 16 f1s on each class and 1 mean f1 
        self.accs = [] # mean accuracy without take imblance into account

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = self.model.predict(x_train)
        y_pred_val = self.model.predict(x_val)
        y_pred_test = self.model.predict(x_test)
        
        y_pred_train_idx = np.argmax(y_pred_train,axis = -1)
        y_pred_val_idx = np.argmax(y_pred_val,axis = -1)
        y_pred_test_idx = np.argmax(y_pred_test,axis = -1)
                
        auc = [] #[train,val,test]
        auc_train = roc_auc_score(y_train, y_pred_train,'weighted')
        auc.append(auc_train)
        auc_val = roc_auc_score(y_val, y_pred_val,'weighted')
        auc.append(auc_val)
        auc_test = roc_auc_score(y_test, y_pred_test,'weighted')
        auc.append(auc_test)
        self.aucs.append(auc)
        print("Weighted AUCs:",auc)
        
        recall = []
        recall_all = recall_score(y_train_idx,y_pred_train_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_train_idx,y_pred_train_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_train_idx,y_pred_train_idx,average='macro')
        recall.append(recall_mean)
        recall_all = recall_score(y_val_idx,y_pred_val_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_val_idx,y_pred_val_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_val_idx,y_pred_val_idx,average='macro')
        recall.append(recall_mean)
        recall_all = recall_score(y_test_idx,y_pred_test_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='macro')
        recall.append(recall_mean)
        self.recalls.append(recall)
        print("weighted average Recalls:",recall[1],recall[4],recall[7])
        print("unweighted average Recalls (macro):",recall[2],recall[5],recall[8])
        
        precision = []
        precision_all = precision_score(y_train_idx,y_pred_train_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_train_idx,y_pred_train_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_train_idx,y_pred_train_idx,average='macro')
        precision.append(precision_mean)
        precision_all = precision_score(y_val_idx,y_pred_val_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_val_idx,y_pred_val_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_val_idx,y_pred_val_idx,average='macro')
        precision.append(precision_mean)
        precision_all = precision_score(y_test_idx,y_pred_test_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='macro')
        precision.append(precision_mean)
        self.precisions.append(precision)
        print("weighted average Precisions:",precision[1],precision[4],precision[7])
        print("unweighted average Precisions(macro):",precision[2],precision[5],precision[8])
        
        f1 = []
        f1_all = f1_score(y_train_idx,y_pred_train_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='macro')
        f1.append(f1_mean)
        f1_all = f1_score(y_val_idx,y_pred_val_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_val_idx,y_pred_val_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_val_idx,y_pred_val_idx,average='macro')
        f1.append(f1_mean)
        f1_all = f1_score(y_test_idx,y_pred_test_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='macro')
        f1.append(f1_mean)
        self.f1s.append(f1)
        print("weighted average F1s:",f1[1],f1[4],f1[7])
        print("unweighted average F1s(macro):",f1[2],f1[5],f1[8])
        
        loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
        self.accs.append(acc)
        print("Test Categorical Accuracy:",acc)
                
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'metrics_save/crnn_aucs.pkl'),'wb'))
pickle.dump(histories.recalls,open(os.path.join(path,'metrics_save/crnn_recalls.pkl'),'wb'))
pickle.dump(histories.precisions,open(os.path.join(path,'metrics_save/crnn_precisions.pkl'),'wb'))
pickle.dump(histories.f1s,open(os.path.join(path,'metrics_save/crnn_f1s.pkl'),'wb'))
pickle.dump(histories.accs,open(os.path.join(path,'metrics_save/crnn_accs.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'metrics_save/crnn_history.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 96, 1360, 64)      640       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 96, 1360, 64)      256       
_________________________________________________________________
act1 (Activation)            (None, 96, 1360, 64)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 680, 64)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 680, 128)      73856     
_________________________________________________________________
bn2 (BatchNormalization)     (None, 48, 680, 128)      512       
_________________________________________________________________
act2 (Activation)            (None, 48, 680, 128)      0         
__________

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


weighted average F1s: 0.5453544806774784 0.5438579120280509 0.5175797745370245
unweighted average F1s(macro): 0.2852599383683506 0.2852599383683506 0.2852599383683506
Test Categorical Accuracy: 0.5664852255170304
Epoch 2/30
19909/19909 [==============================] - 459s 23ms/step - loss: 1.2910 - categorical_accuracy: 0.6122 - val_loss: 1.3111 - val_categorical_accuracy: 0.5954
Weighted AUCs: [0.9037243006918845, 0.9028138121005853, 0.8970642171902545]
weighted average Recalls: 0.5981214526093727 0.5954472843450479 0.578538102643857
unweighted average Recalls (macro): 0.27112109502531767 0.27112109502531767 0.27112109502531767
weighted average Precisions: 0.6036399837632257 0.5838515942732995 0.5489423524421031
unweighted average Precisions(macro): 0.37753490201979434 0.37753490201979434 0.37753490201979434
weighted average F1s: 0.5508942979952743 0.5401261681328033 0.5197114398613617
unweighted average F1s(macro): 0.2804164398964806 0.2804164398964806 0.2804164398964806
Test Cate

In [1]:
#  4 CNNs + GridRNN + output  (FMA)
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 30
input_shape = (96,1360,1)
input_melS = Input(input_shape,name='main_input')

conv1 = Conv2D(64,(3,3),padding='same',name='conv_1')(input_melS)
bn1 = BatchNormalization(name='bn_1')(conv1)
act1 = Activation('elu',name='act_1')(bn1)
pool1 = MaxPooling2D((2,4),padding='same',name='pool_1')(act1)

conv2 = Conv2D(128,(3,3),padding='same',name='conv_2')(pool1)
bn2 = BatchNormalization(name='bn_2')(conv2)
act2 = Activation('elu',name='act_2')(bn2)
pool2 = MaxPooling2D((2,4),padding='same',name='pool_2')(act2)

conv3 = Conv2D(128,(3,3),padding='same',name='conv_3')(pool2)
bn3 = BatchNormalization(name='bn_3')(conv3)
act3 = Activation('elu',name='act_3')(bn3)
pool3 = MaxPooling2D((2,4),padding='same',name='pool_3')(act3)

conv4 = Conv2D(128,(3,3),padding='same',name='conv_4')(pool3)
bn4 = BatchNormalization(name='bn_4')(conv4)
act4 = Activation('elu',name='act_4')(bn4)
pool4 = MaxPooling2D((2,4),padding='same',name='pool_4')(act4)

gru1_in = Reshape((6, 128*6),name='gru1_in')(pool4)
gru1 = GRU(32,name='gru1')(gru1_in)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru2_in_perm')(pool4)
gru2_in = Reshape((6, 128*6),name='gru2_in')(gru2_in_perm)
gru2 = GRU(32,name='gru2')(gru2_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2},name='gru3_in_perm')(pool4)
gru3_in = Reshape((6, 128*6),name='gru3_in')(gru3_in_perm)
gru3 = GRU(32,name='gru3')(gru3_in)

gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru4_in_perm')(gru3_in_perm)
gru4_in = Reshape((6, 128*6),name='gru4_in')(gru4_in_perm)
gru4 = GRU(32,name='gru4')(gru4_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1},name='gru5_in_perm')(pool4)
gru5_in = Reshape((6, 128*6),name='gru5_in')(gru5_in_perm)
gru5 = GRU(32,name='gru5')(gru5_in)

gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru6_in_perm')(gru5_in_perm)
gru6_in = Reshape((6, 128*6),name='gru6_in')(gru6_in_perm)
gru6 = GRU(32,name='gru6')(gru6_in)

gru7_in_perm = Lambda(K.reverse,arguments={'axes':2},name='gru7_in_perm')(gru5_in_perm)
gru7_in = Reshape((6, 128*6),name='gru7_in')(gru7_in_perm)
gru7 = GRU(32,name='gru7')(gru7_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1},name='gru8_in_perm')(gru6_in_perm)
gru8_in = Reshape((6, 128*6),name='gru8_in')(gru8_in_perm)
gru8 = GRU(32,name='gru8')(gru8_in)

gru = Concatenate(name='gru_conc')([gru1,gru2,gru3,gru4,gru5,gru6,gru7,gru8])
gru_dropout = Dropout(0.5,name='gru_conc_dropout')(gru)
fc = Dense(64, activation='elu',name='fc')(gru_dropout)
pred = Dense(16, activation='softmax',name='pred')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(),metrics=['categorical_accuracy'])

filepath='C:/DT/model_save/fma/cGrnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
y_train_idx = np.argmax(y_train,axis = -1)
x_val = np.load(path+'\\x_val.npy')
y_val = np.load(path+'\\val_y.npy')
y_val_idx = np.argmax(y_val,axis = -1)
x_test = np.load(path+'\\x_test.npy')
y_test = np.load(path+'\\test_y.npy')
y_test_idx = np.argmax(y_test,axis = -1)

from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        # all evaluation metrics include train, val, test except accs
        self.aucs = [] # weighted AUC
        self.recalls = [] # 16 recalls on each class and 1 mean recall
        self.precisions = [] # 16 precisions on each class and 1 mean precision
        self.f1s = [] # 16 f1s on each class and 1 mean f1 
        self.accs = [] # mean accuracy without take imblance into account

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = self.model.predict(x_train)
        y_pred_val = self.model.predict(x_val)
        y_pred_test = self.model.predict(x_test)
        
        y_pred_train_idx = np.argmax(y_pred_train,axis = -1)
        y_pred_val_idx = np.argmax(y_pred_val,axis = -1)
        y_pred_test_idx = np.argmax(y_pred_test,axis = -1)
                
        auc = [] #[train,val,test]
        auc_train = roc_auc_score(y_train, y_pred_train,'weighted')
        auc.append(auc_train)
        auc_val = roc_auc_score(y_val, y_pred_val,'weighted')
        auc.append(auc_val)
        auc_test = roc_auc_score(y_test, y_pred_test,'weighted')
        auc.append(auc_test)
        self.aucs.append(auc)
        print("Weighted AUCs:",auc)
        
        recall = []
        recall_all = recall_score(y_train_idx,y_pred_train_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_train_idx,y_pred_train_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_train_idx,y_pred_train_idx,average='macro')
        recall.append(recall_mean)
        recall_all = recall_score(y_val_idx,y_pred_val_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_val_idx,y_pred_val_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_val_idx,y_pred_val_idx,average='macro')
        recall.append(recall_mean)
        recall_all = recall_score(y_test_idx,y_pred_test_idx,average=None)
        recall.append(recall_all)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='weighted')
        recall.append(recall_mean)
        recall_mean = recall_score(y_test_idx,y_pred_test_idx,average='macro')
        recall.append(recall_mean)
        self.recalls.append(recall)
        print("weighted average Recalls:",recall[1],recall[4],recall[7])
        print("unweighted average Recalls (macro):",recall[2],recall[5],recall[8])
        
        precision = []
        precision_all = precision_score(y_train_idx,y_pred_train_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_train_idx,y_pred_train_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_train_idx,y_pred_train_idx,average='macro')
        precision.append(precision_mean)
        precision_all = precision_score(y_val_idx,y_pred_val_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_val_idx,y_pred_val_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_val_idx,y_pred_val_idx,average='macro')
        precision.append(precision_mean)
        precision_all = precision_score(y_test_idx,y_pred_test_idx,average=None)
        precision.append(precision_all)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='weighted')
        precision.append(precision_mean)
        precision_mean = precision_score(y_test_idx,y_pred_test_idx,average='macro')
        precision.append(precision_mean)
        self.precisions.append(precision)
        print("weighted average Precisions:",precision[1],precision[4],precision[7])
        print("unweighted average Precisions(macro):",precision[2],precision[5],precision[8])
        
        f1 = []
        f1_all = f1_score(y_train_idx,y_pred_train_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_train_idx,y_pred_train_idx,average='macro')
        f1.append(f1_mean)
        f1_all = f1_score(y_val_idx,y_pred_val_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_val_idx,y_pred_val_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_val_idx,y_pred_val_idx,average='macro')
        f1.append(f1_mean)
        f1_all = f1_score(y_test_idx,y_pred_test_idx,average=None)
        f1.append(f1_all)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='weighted')
        f1.append(f1_mean)
        f1_mean = f1_score(y_test_idx,y_pred_test_idx,average='macro')
        f1.append(f1_mean)
        self.f1s.append(f1)
        print("weighted average F1s:",f1[1],f1[4],f1[7])
        print("unweighted average F1s(macro):",f1[2],f1[5],f1[8])
        
        loss, acc = self.model.evaluate(x_test, y_test, verbose=0)
        self.accs.append(acc)
        print("Test Categorical Accuracy:",acc)
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[histories,checkpoint])

import pickle
import os
pickle.dump(histories.aucs,open(os.path.join(path,'metrics_save/cGrnn_aucs.pkl'),'wb'))
pickle.dump(histories.recalls,open(os.path.join(path,'metrics_save/cGrnn_recalls.pkl'),'wb'))
pickle.dump(histories.precisions,open(os.path.join(path,'metrics_save/cGrnn_precisions.pkl'),'wb'))
pickle.dump(histories.f1s,open(os.path.join(path,'metrics_save/cGrnn_f1s.pkl'),'wb'))
pickle.dump(histories.accs,open(os.path.join(path,'metrics_save/cGrnn_accs.pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(path,'metrics_save/cGrnn_history.pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 1360, 64) 640         main_input[0][0]                 
__________________________________________________________________________________________________
bn_1 (BatchNormalization)       (None, 96, 1360, 64) 256         conv_1[0][0]                     
__________________________________________________________________________________________________
act_1 (Activation)              (None, 96, 1360, 64) 0           bn_1[0][0]                       
__________________________________________________________________________________________________
pool_1 (Ma

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Test Categorical Accuracy: 0.46656298601469764
Epoch 2/30
19909/19909 [==============================] - 415s 21ms/step - loss: 1.3220 - categorical_accuracy: 0.5930 - val_loss: 1.3177 - val_categorical_accuracy: 0.5915
Weighted AUCs: [0.894544945787742, 0.8905739771089058, 0.884717407211161]
weighted average Recalls: 0.6002812798231956 0.5914536741214057 0.5692068429237948
unweighted average Recalls (macro): 0.3319081494887667 0.3319081494887667 0.3319081494887667
weighted average Precisions: 0.5771790931102986 0.5425203627980586 0.5278120763596411
unweighted average Precisions(macro): 0.35519883561661686 0.35519883561661686 0.35519883561661686
weighted average F1s: 0.5607342552468253 0.5476293738763767 0.5255285121705955
unweighted average F1s(macro): 0.31213042841181626 0.31213042841181626 0.31213042841181626
Test Categorical Accuracy: 0.5692068429237948
Epoch 3/30
19909/19909 [==============================] - 415s 21ms/step - loss: 1.2511 - categorical_accuracy: 0.6145 - val_loss:

In [1]:
#  4 CNNs + GridRNN + output  (FMA)
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 10
input_shape = (96,1360,1)
input_melS = Input(input_shape,name='main_input')

conv1 = Conv2D(64,(3,3),padding='same',name='conv_1')(input_melS)
bn1 = BatchNormalization(name='bn_1')(conv1)
act1 = Activation('elu',name='act_1')(bn1)
pool1 = MaxPooling2D((2,4),padding='same',name='pool_1')(act1)

conv2 = Conv2D(128,(3,3),padding='same',name='conv_2')(pool1)
bn2 = BatchNormalization(name='bn_2')(conv2)
act2 = Activation('elu',name='act_2')(bn2)
pool2 = MaxPooling2D((2,4),padding='same',name='pool_2')(act2)

conv3 = Conv2D(128,(3,3),padding='same',name='conv_3')(pool2)
bn3 = BatchNormalization(name='bn_3')(conv3)
act3 = Activation('elu',name='act_3')(bn3)
pool3 = MaxPooling2D((2,4),padding='same',name='pool_3')(act3)

conv4 = Conv2D(128,(3,3),padding='same',name='conv_4')(pool3)
bn4 = BatchNormalization(name='bn_4')(conv4)
act4 = Activation('elu',name='act_4')(bn4)
pool4 = MaxPooling2D((2,4),padding='same',name='pool_4')(act4)

gru1_in = Reshape((6, 128*6),name='gru1_in')(pool4)
gru1 = GRU(32,name='gru1')(gru1_in)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru2_in_perm')(pool4)
gru2_in = Reshape((6, 128*6),name='gru2_in')(gru2_in_perm)
gru2 = GRU(32,name='gru2')(gru2_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2},name='gru3_in_perm')(pool4)
gru3_in = Reshape((6, 128*6),name='gru3_in')(gru3_in_perm)
gru3 = GRU(32,name='gru3')(gru3_in)

gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru4_in_perm')(gru3_in_perm)
gru4_in = Reshape((6, 128*6),name='gru4_in')(gru4_in_perm)
gru4 = GRU(32,name='gru4')(gru4_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1},name='gru5_in_perm')(pool4)
gru5_in = Reshape((6, 128*6),name='gru5_in')(gru5_in_perm)
gru5 = GRU(32,name='gru5')(gru5_in)

gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)},name='gru6_in_perm')(gru5_in_perm)
gru6_in = Reshape((6, 128*6),name='gru6_in')(gru6_in_perm)
gru6 = GRU(32,name='gru6')(gru6_in)

gru7_in_perm = Lambda(K.reverse,arguments={'axes':2},name='gru7_in_perm')(gru5_in_perm)
gru7_in = Reshape((6, 128*6),name='gru7_in')(gru7_in_perm)
gru7 = GRU(32,name='gru7')(gru7_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1},name='gru8_in_perm')(gru6_in_perm)
gru8_in = Reshape((6, 128*6),name='gru8_in')(gru8_in_perm)
gru8 = GRU(32,name='gru8')(gru8_in)

gru = Concatenate(name='gru_conc')([gru1,gru2,gru3,gru4,gru5,gru6,gru7,gru8])
gru_dropout = Dropout(0.5,name='gru_conc_dropout')(gru)
fc = Dense(64, activation='elu',name='fc')(gru_dropout)
pred = Dense(16, activation='sigmoid',name='pred')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam(),metrics=['acc'])

filepath='C:/DT/model_save/fma/cGrnn/'+"{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

path = 'C:/DT/fma_medium'
x_train = np.load(path+'\\x_train.npy')
y_train = np.load(path+'\\train_y.npy')
x_val = np.load(path+'\\x_val.npy')
y_val = np.load(path+'\\val_y.npy')

history = model.fit(x_train, y_train, epochs = epochs, validation_data=(x_val,y_val), callbacks=[checkpoint])

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 1360, 64) 640         main_input[0][0]                 
__________________________________________________________________________________________________
bn_1 (BatchNormalization)       (None, 96, 1360, 64) 256         conv_1[0][0]                     
__________________________________________________________________________________________________
act_1 (Activation)              (None, 96, 1360, 64) 0           bn_1[0][0]                       
__________________________________________________________________________________________________
pool_1 (Ma

In [1]:
# Extracting features from last fc layer (crnn)
# from keras.models import Model
# import keras
# import numpy as np

# model = keras.models.load_model('C:/DT/model_save/fma/crnn/13-1.09.hdf5')
# fc = model.get_layer('gru2').output

# features = Model(inputs=model.input, outputs=fc)
# features.summary()

# path = 'C:/DT/fma_medium'
# x_test = np.load(path+'\\x_test.npy')
# feats = features.predict(x_test)
# print(feats.shape)
# np.save(path+'/test_feats_crnn_13',feats)

# Extracting features from last fc layer (cGrnn)
from keras.models import Model
import keras
import tensorflow
import numpy as np

model = keras.models.load_model('C:/DT/model_save/fma/cGrnn/29-1.17.hdf5',custom_objects={
                                                'tf':tensorflow})
fc = model.get_layer('fc').output

features = Model(inputs=model.input, outputs=fc)
features.summary()

path = 'C:/DT/fma_medium'
x_test = np.load(path+'\\x_test.npy')
feats = features.predict(x_test)
print(feats.shape)
np.save(path+'/test_feats_cGrnn_29',feats)

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 1360, 64) 640         main_input[0][0]                 
__________________________________________________________________________________________________
bn_1 (BatchNormalization)       (None, 96, 1360, 64) 256         conv_1[0][0]                     
__________________________________________________________________________________________________
act_1 (Activation)              (None, 96, 1360, 64) 0           bn_1[0][0]                       
__________________________________________________________________________________________________
pool_1 (Ma